In [3]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

from scipy import stats
# from scipy.interpolate import \
#     interp2d, griddata, RegularGridInterpolator, RectBivariateSpline, \
#     NearestNDInterpolator, LinearNDInterpolator

from vivarium import Artifact # This is causing my kernel to die!!! Memory leak?
import gbd_mapping as gbd
from db_queries import get_ids
from get_draws.api import get_draws

from vivarium_helpers.id_helper import *

# Add path to vivarium_research_lsff to sys.path so imports will work from sibling folders
import sys, os.path
# Note: Assuming pwd = vivarium_research_lsff/nanosim_models
vivarium_research_lsff_path = os.path.abspath("..")
print(vivarium_research_lsff_path)
sys.path.append(vivarium_research_lsff_path)

import lbwsg
import lbwsg_boe # Requires path to vivarium_research_lsff to be in sys.path
import data_processing # Requires path to vivarium_research_lsff to be in sys.path
import lsff_interventions
import demography
import test_lbwsg
import lbwsg_plots
import functions_for_all_nutrients as ffan


# Assumes the path to vivarium_research_lsff is in sys.path
# from gbd_data_summary import pull_gbd2019_data as pgd, rank_countries_by_stunting as rbs

!date
!whoami
!pwd

/ihme/homes/ndbs/vivarium_research_lsff
Thu Apr 15 17:20:11 PDT 2021
ndbs
/ihme/homes/ndbs/vivarium_research_lsff/nanosim_models


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
draws=range(50)
calc = lbwsg_boe.IronBirthweightCalculator.from_parameters(
    'Nigeria',
    'wheat flour',
    draws,
)
calc

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/tables/atom.py:1224: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return pickle.loads(array.tostring())


In [7]:
ages = [4/365, 14/365]
num_simulants=10_000
%timeit calc.initialize_population_tables(num_simulants, ages)

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


466 ms ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit calc.assign_lbwsg_exposure()

601 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit calc.assign_iron_treatment_deleted_birthweights()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.26 s ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit calc.assign_iron_treated_birthweights()

2.47 s ± 72.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit calc.assign_lbwsg_relative_risks()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


20.2 s ± 221 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit calc.calculate_potential_impact_fraction()

16.2 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%timeit calc.calculate_averted_dalys()

55.2 ms ± 437 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%timeit calc.do_back_of_envelope_calculation(10_000, ages)

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


26 s ± 446 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
20.2/26.4

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.7651515151515151

In [17]:
0.7651515151515151/(1-0.7651515151515151)

3.258064516129032

In [18]:
%timeit calc.do_back_of_envelope_calculation(20_000, ages)

50.8 s ± 786 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
